In [117]:
from google.colab import drive
drive.mount('/content/drive/')

PATH = '/content/drive/My Drive/dataset/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
# while success:
#   if not cv2.imwrite('image_{}.jpg'.format(cnt), image):
#     print('error')
#   success, image = vidcap.read()   
#   print('Saved image #', cnt)
#   cnt = cnt + 1



> Video is splitted into images and then images are converted into **LAB** color space



In [0]:
import os, cv2
import numpy as np
from skimage.color import rgb2lab

os.chdir(PATH)

vidcap = cv2.VideoCapture('peacock.mp4')
success, image = vidcap.read()
cnt = 1

os.chdir(PATH + 'video_images')
video_images = []

DIM = (256, 256)
while success:
  video_images.append(cv2.resize(image, DIM, interpolation = cv2.INTER_AREA) * 1. / 255)
  success, image = vidcap.read()   
  print('Saved image #', cnt)
  cnt = cnt + 1


input_images = [(rgb2lab(img)[:,:,0]) for img in video_images]
target_images = [(rgb2lab(img)[:,:,1:]) for img in video_images]

# print(len(input_images))
# print(input_images[0].shape)

# print(len(target_images))
# print(target_images[0].shape)

input_images = np.array(input_images)
target_images = np.array(target_images)
input_images = input_images.reshape(input_images.shape + (1, ))

> Method to combine **L a b** channels and convert to **RGB**



In [0]:
from skimage.color import lab2rgb
import matplotlib.pyplot as plt

def lab_to_rgb(input, output):
  rgb_images = []

  for x, y in zip(input, output):
    out = np.zeros((256, 256, 3))
    out[:,:,0] = x[:,:,0]
    out[:,:,1:] = y
    rgb_images.append(lab2rgb(out))

  return rgb_images

> Method to plot specific **images array**

In [0]:
def imshow_rgb(images):
  for img in images:
    plt.imshow(img)
    plt.show()


> Model prediction on **L** channel



In [0]:
import tensorflow as tf

model = tf.keras.models.load_model(PATH + 'autoencoder700.model')
result = model.predict(input_images)
result *= 128



# imshow_rgb(lab_to_rgb(input_images, result))


In [122]:
from google.colab.patches import cv2_imshow
images_from_result = lab_to_rgb(input_images, result)

os.chdir(PATH)

def generate_video(images): 
  video_name = 'peacock_video_output.avi'
  height, width = (256, 256)  

  fourcc = cv2.VideoWriter_fourcc('X','V','I','D')
  video = cv2.VideoWriter(video_name, fourcc, 30, (height, width))  

  # appending the images to the video one by one 
  for image in images:
    video.write(np.uint8(image * 255))
    
  # deallocating memories taken for window creation 
  cv2.destroyAllWindows()  
  video.release()  # releasing the video generated

generate_video(images_from_result)

/usr/local/lib/python3.6/dist-packages/skimage/color/colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 1 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.6/dist-packages/skimage/color/colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 2 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
